In [2]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
from datetime import datetime

/Users/qhall/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Page config
st.set_page_config(
    page_title="Fantasy Football Analytics",
    page_icon="🏈",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Title and header
st.title("🏈 Fantasy Football Team Analytics")
st.markdown("---")

# Sidebar for navigation
st.sidebar.title("Dashboard Navigation")
page = st.sidebar.selectbox(
    "Choose a page:",
    ["Team Overview", "Player Analysis", "Matchup Predictor", "Season Stats"]
)

# Sample data - replace with your API calls
@st.cache_data
def load_team_data():
    # This is where you'd call your API
    # For now, using sample data
    return pd.DataFrame({
        'Player': ['Josh Allen', 'Christian McCaffrey', 'Tyreek Hill', 'Travis Kelce'],
        'Position': ['QB', 'RB', 'WR', 'TE'],
        'Points': [342.5, 289.2, 267.8, 198.3],
        'Games': [17, 14, 16, 15],
        'PPG': [20.1, 20.7, 16.7, 13.2]
    })

@st.cache_data
def load_weekly_scores():
    # Sample weekly scores data
    weeks = list(range(1, 15))
    return pd.DataFrame({
        'Week': weeks,
        'Your_Team': [145, 162, 98, 178, 134, 156, 189, 123, 167, 145, 134, 178, 156, 189],
        'League_Average': [135, 142, 128, 155, 148, 162, 171, 139, 158, 149, 144, 165, 151, 163]
    })

# Main content based on page selection
if page == "Team Overview":
    st.header("📊 Team Overview")
    
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric("Total Points", "2,145", "+45 vs avg")
    with col2:
        st.metric("Current Rank", "3rd", "+2 positions")
    with col3:
        st.metric("Wins", "9", "+2 vs expected")
    with col4:
        st.metric("Points Per Game", "154.3", "+12.1 vs avg")
    
    # Weekly performance chart
    weekly_data = load_weekly_scores()
    
    fig = px.line(weekly_data, x='Week', y=['Your_Team', 'League_Average'], 
                  title='Weekly Performance vs League Average',
                  labels={'value': 'Points', 'variable': 'Team'})
    st.plotly_chart(fig, use_container_width=True)
    
    # Team roster performance
    st.subheader("🏆 Top Performers")
    team_data = load_team_data()
    
    col1, col2 = st.columns(2)
    
    with col1:
        # Points bar chart
        fig_bar = px.bar(team_data, x='Player', y='Points', 
                        title='Total Points by Player',
                        color='Position')
        st.plotly_chart(fig_bar, use_container_width=True)
    
    with col2:
        # PPG comparison
        fig_ppg = px.scatter(team_data, x='Games', y='PPG', 
                           size='Points', hover_name='Player',
                           title='Points Per Game vs Games Played',
                           color='Position')
        st.plotly_chart(fig_ppg, use_container_width=True)

elif page == "Player Analysis":
    st.header("👤 Player Analysis")
    
    team_data = load_team_data()
    selected_player = st.selectbox("Select a player:", team_data['Player'].tolist())
    
    player_info = team_data[team_data['Player'] == selected_player].iloc[0]
    
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Position", player_info['Position'])
    with col2:
        st.metric("Total Points", f"{player_info['Points']:.1f}")
    with col3:
        st.metric("PPG", f"{player_info['PPG']:.1f}")
    
    # Player performance trends (sample data)
    weeks = list(range(1, 15))
    performance = [20, 15, 25, 8, 22, 18, 26, 12, 24, 19, 16, 23, 21, 27]
    
    player_df = pd.DataFrame({
        'Week': weeks,
        'Points': performance
    })
    
    fig = px.line(player_df, x='Week', y='Points', 
                  title=f'{selected_player} - Weekly Performance')
    st.plotly_chart(fig, use_container_width=True)

elif page == "Matchup Predictor":
    st.header("🔮 Matchup Predictor")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Your Team")
        st.write("Projected Points: **147.5**")
        st.write("Confidence: **78%**")
        
    with col2:
        st.subheader("Opponent")
        opponent = st.selectbox("Select opponent:", ["Team Alpha", "Team Beta", "Team Gamma"])
        st.write("Projected Points: **142.3**")
        st.write("Win Probability: **62%**")
    
    # Matchup visualization
    matchup_data = pd.DataFrame({
        'Position': ['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'TE', 'FLEX', 'K', 'DST'],
        'Your_Team': [22, 18, 12, 16, 14, 11, 13, 9, 8],
        'Opponent': [19, 16, 14, 17, 15, 10, 11, 8, 7]
    })
    
    fig = px.bar(matchup_data, x='Position', y=['Your_Team', 'Opponent'],
                 title='Position-by-Position Matchup Projection',
                 barmode='group')
    st.plotly_chart(fig, use_container_width=True)

elif page == "Season Stats":
    st.header("📈 Season Statistics")
    
    # League standings table
    standings = pd.DataFrame({
        'Rank': [1, 2, 3, 4, 5],
        'Team': ['Team Alpha', 'Team Beta', 'Your Team', 'Team Gamma', 'Team Delta'],
        'Wins': [11, 10, 9, 8, 7],
        'Losses': [3, 4, 5, 6, 7],
        'Points For': [2190, 2156, 2145, 2089, 2034],
        'Points Against': [1987, 2023, 2067, 2156, 2178]
    })
    
    st.subheader("League Standings")
    st.dataframe(standings, use_container_width=True)
    
    # Points distribution
    st.subheader("Points Distribution")
    fig = px.histogram(standings, x='Points For', nbins=10,
                      title='League Points Distribution')
    st.plotly_chart(fig, use_container_width=True)

# Sidebar info
st.sidebar.markdown("---")
st.sidebar.info(
    "This dashboard updates automatically with your latest fantasy football data. "
    "Connect your API endpoints to get real-time analytics!"
)

# Footer
st.markdown("---")
st.markdown("*Last updated: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "*")

2025-09-06 16:06:54.932 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 16:06:54.935 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 16:06:54.970 
  command:

    streamlit run /Users/qhall/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-06 16:06:54.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 16:06:54.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 16:06:54.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 16:06:54.972 Thread 'MainThread': missing ScriptRunContext! This warning can be ign

DeltaGenerator()